In [1]:
# from https://www.kaggle.com/ievgenvp/keras-flow-from-directory-on-python

# Sequential is a basic layer-by-layer building approach. Good for CNN, but more advanced topologies don't work with it
from keras.models import Sequential

# Conv2D - explained earlier
# MaxPooling - explained earlier
from keras.layers import Conv2D, MaxPooling2D

# Dropout - randomly exclude inputs, prevents overfitting.
# Flatten - take an n x m matrix and change it to an nm x 1 matrix
# Dense - All inputs connected to all outputs.
from keras.layers import Dropout, Flatten, Dense

from keras.preprocessing.image import ImageDataGenerator



/Users/benwendt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/benwendt/anaconda3/lib/python3.6/site-packages/keras/backend/__init__.py:27: ResourceWarning: unclosed file <_io.TextIOWrapper name='/Users/benwendt/.keras/keras.json' mode='r' encoding='UTF-8'>
  _config = json.load(open(_config_path))
Using TensorFlow backend.
/Users/benwendt/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:766: DeprecationWarning: builtin type EagerTensor has no __module__ attribute
  EagerTensor = c_api.TFE_Py_InitEagerTensor(_EagerTensorBase)
/Users/benwendt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/benwendt/anaconda3/lib/python3.6/site-packages/tensorflow/python/u

In [ ]:
# place to put some parameters
batch_size = 20
num_val_samples = 137
steps = 2295*1.4/batch_size

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2, # random application of shearing
    zoom_range = 0.2,
    horizontal_flip = True) # randomly flipping half of the images horizontally

# data generator for test set
test_datagen = ImageDataGenerator(rescale = 1./255)

In [2]:
# generator for reading train data from folder
# class_mode - needs to match the shape of output
train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size = (256, 256),
    color_mode = 'rgb',
    batch_size = batch_size,
    class_mode = 'binary')

# generator for reading validation data from folder
validation_generator = test_datagen.flow_from_directory(
    'data/valid',
    target_size = (256, 256),
    color_mode = 'rgb',
    batch_size = batch_size,
    class_mode = 'binary')

# generator for reading test data from folder
test_generator = test_datagen.flow_from_directory(
    'data/test',
    target_size = (256, 256),
    color_mode = 'rgb',
    batch_size = 1,
    class_mode = 'binary',
    shuffle = False)

Found 139 images belonging to 4 classes.
Found 38 images belonging to 4 classes.
Found 19 images belonging to 4 classes.


In [3]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape = (256, 256, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3)))

model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3,3)))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

# relu - a linear activation with a cutoff.
# sigmoid - a non-linear activation

model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 84, 84, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 82, 82, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

/Users/benwendt/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [4]:
# binary cross entropy - the mmain loss function used for classification
# rmsprop - a way of varying the step size toward the minimum of the loss function
model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

model.fit_generator(train_generator,
                    steps_per_epoch = steps,
                    epochs = 6,
                    validation_data = validation_generator,
                    validation_steps = num_val_samples/batch_size)


Epoch 1/6
161/160 [==============================] - 232s 1s/step - loss: -5.1836 - acc: 0.5381 - val_loss: -4.7731 - val_acc: 0.4552
Epoch 2/6
161/160 [==============================] - 249s 2s/step - loss: -6.5668 - acc: 0.6680 - val_loss: -6.2617 - val_acc: 0.5455
Epoch 3/6
161/160 [==============================] - 252s 2s/step - loss: -6.7581 - acc: 0.6905 - val_loss: -6.0483 - val_acc: 0.5299
Epoch 4/6
161/160 [==============================] - 202s 1s/step - loss: -6.9456 - acc: 0.7050 - val_loss: -5.6573 - val_acc: 0.5530
Epoch 5/6
161/160 [==============================] - 182s 1s/step - loss: -6.9625 - acc: 0.7079 - val_loss: -4.8318 - val_acc: 0.5970
Epoch 6/6
161/160 [==============================] - 173s 1s/step - loss: -6.9356 - acc: 0.7164 - val_loss: -5.8373 - val_acc: 0.5833
